# LLM 만들어보기

LLM 구축부터 미세튜닝까지의 전 과정을 다음 4단계로 요약하여 진행합니다.

In [43]:
!pip install tiktoken

In [44]:
from importlib.metadata import version

print('파이토치 버전 : ', version('torch'))
print('tiktoken version : ' , version('tiktoken'))

파이토치 버전 :  2.9.0+cu126
tiktoken version :  0.12.0


In [45]:
file_path = "/content/drive/MyDrive/study/LLM/the-verdict.txt"

## **텍스트 토큰화하기**

* 사용할 텍스트 데이터는 이디스 워튼의 단편 소설 "The Verdict" 가져오기

In [46]:
import os
import requests

if not os.path.exists(file_path):
  url = (
      "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
  )
  file_path = "the-verdict.txt"
  raw_text = ""
  response = requests.get(url, timeout=30)
  response.raise_for_status()
  with open(file_path, "wb") as f:
    f.write(response.content)


In [47]:

with open("/content/drive/MyDrive/study/LLM/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("총 문자 개수:", len(raw_text))
print(raw_text[:99])

총 문자 개수: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


* LLM을 위해 텍스트를 토큰화, 임베딩
* 간단한 토크나이저 만들고 텍스트에 적용

* 공백과 쉼표, 마침표, 구둣점등으로 문자처리 그리고 빈문자열 삭제


In [48]:
import re

text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


토큰화 원시 텍스트에 적용

In [49]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

print("총 개수 :",len(preprocessed))


['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
총 개수 : 4690


*이탤릭체 텍스트*# 새 섹션

## **토큰을 토큰 ID로 변환하기**



* 토큰을 고유한 토큰으로 구성된 어휘 사전을 만들기


In [50]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)
vocab = {token:integer for integer,token in enumerate(all_words)}

1130


* 어휘사전을 만들었으니 50개 항목확인

In [51]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


### 토큰화 하는 클래스 생성

In [52]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text) # 'hello,. world'

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # 구둣점 문자 앞의 공백을 삭제합니다.
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

'''
encode 함수는 텍스트를 토큰 ID로 바꿉니다.
decode 함수는 토큰 ID를 텍스트로 바꿉니다.
'''

'\nencode 함수는 텍스트를 토큰 ID로 바꿉니다.\ndecode 함수는 토큰 ID를 텍스트로 바꿉니다.\n'

### 토크나이저를 사용해 텍스트를 정수로 인코딩(토큰화)

* 굵은 텍스트 이정수를 나중에 임베딩하여 LLM의 입력에 사용가능

In [53]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


* 정수를 텍스트로 다시 디코딩하는 경우

In [54]:
print(tokenizer.decode(ids))
print(tokenizer.decode(tokenizer.encode(text)))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


## **특수 문맥 토큰 추가하기**

* 알지 못하는 단어와 텍스트의 끝을 알리는 "특수" 토큰을 추가하면 좋다

In [55]:
#어휘사전에 등록되지 않은 경우 오류 발생
tokenizer = SimpleTokenizerV1(vocab)
try:
    # 원본 코드
    text = "Hello, do you like tea. Is this-- a test?"
    tokenizer.encode(text)
except KeyError as e:
    # KeyError 발생 시 처리 로직
    print(f"사전(Dictionary)에서 찾을 수 없는 문자열 발견: {e}")
    # 추가 처리 옵션 (예: 특수문자 무시, 기본 ID 할당 등)
    # 예: 특수문자를 무시하고 빈 리스트 반환
    print("특수문자 무시 후 재처리 시도")

    # (선택사항) 대체 ID 사용 예시
    default_id = 0  # 사용자 정의 기본 ID
    encoded_text = []
    for char in text:
        encoded_text.append(tokenizer.str_to_int.get(char, default_id))
    print(f"대체된 인코딩 결과: {encoded_text}")

사전(Dictionary)에서 찾을 수 없는 문자열 발견: 'Hello'
특수문자 무시 후 재처리 시도
대체된 인코딩 결과: [0, 0, 0, 0, 0, 5, 0, 310, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 970, 0, 115, 7, 0, 53, 850, 0, 970, 0, 0, 850, 0, 0, 0, 115, 0, 970, 0, 850, 970, 10]


* 이를 처리하기 위해 알지 못하는 단어를 표현하는 "<|unk|>" 같은 특수 토큰을 어휘 사전에 추가
* 어휘사전을 준비했으므로 GPT-2 훈련에서 텍스트의 끝을 나타내기 위해 사용된 "<|endoftext|>" 토큰을 추가

In [56]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

새로운 <unk> 토큰을 사용할 때와 방법을 알려 주기 위해 토크나이저를 수정

In [57]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # 구둣점 문자 앞의 공백을 삭제합니다.
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [58]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [59]:
print("토큰화된 :", tokenizer.encode(text))
print("디코드된 :", tokenizer.decode(tokenizer.encode(text)))

토큰화된 : [1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
디코드된 : <|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


## 바이트 페어 인코딩

* GPT-2는 바이트 페어 인코딩(BPE) 토크나이저 사용
* 어휘사전에 없는 단어를 더 작은 부분단어나 개별 문자로 분할하여 처리할 수 있다.
* BPE 토크나이저는 알지 못하는 단어를 부분단어나 개별 문자로 분할한다.

In [60]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [61]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [62]:
tokenizer.special_tokens_set

{'<|endoftext|>'}

In [63]:
print(tokenizer.encode(text, allowed_special='all'))

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [64]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


## 슬라이딩 윈도로 데이터 샘플링하기

In [65]:
with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


* 텍스트 청크에 대해 입력과 타깃이 있는지 확인
* 모델이 다음 단어를 예측하고 타깃은 오른쪽으로 한 토큰 이동한 입력

In [66]:
enc_sample = enc_text[50:]
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


* 한 번에 하나씩 예측은 다음과 같이 수행

In [67]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [68]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


* 데이터셋을 순회하면서 입력과 타깃을 반환하는  간단한 데이터 로더 구현
* 데이터 셋과 입력 텍스트 데이터셋에서 청크를 추출하는 데이터 로더를 만든다.

In [69]:
import torch
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # 전체 텍스트를 토큰화합니다.
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "토큰화된 입력의 개수는 적어도 max_length+1과 같아야 합니다."

        # 슬라이딩 윈도를 사용해 책을 max_length 길이의 중첩된 시퀀스로 나눕니다.
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [70]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # 토크나이저를 초기화합니다.
    tokenizer = tiktoken.get_encoding("gpt2")

    # 데이터셋을 만듭니다.
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # 데이터 로더를 만듭니다.
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

* 문맥 크기를 4와 배치 크기 1로 데이터 로더를 테스트한 결과

In [71]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


* 배치 출력도 만들수 있다.
* 배치 간에 중첩이 있으면 과대적합이 증가될 수 있으므로 스트라이드를 증가

In [72]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("입력:\n", inputs)
print("\n타깃:\n", targets)

입력:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

타깃:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 토큰 임베딩 만들기

* 임베딩 층을 사용해 토큰을 연속적일 벡터 표현으로 임베딩한다.
* 이런 임베딩 층은 LLM의 일부이며, 모델 훈련 과정에서 업데이트 된다.

In [73]:
input_ids = torch.tensor([2, 3, 5, 1])

* 테스트 : 어휘사전에는 6개 단어만 있고, 임베딩 크기는 3인 경우

결과적으로 6x3 가중치 행렬이 만들어진다.

In [74]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


## 단어 위치 인코딩하기

* 임베딩 층은 토큰 ID를 입력 시퀀스에서 어떤 위치에 있던지 상관없이 동일한 벡터 표현으로 바꾼다.
* 위치 임베딩을 토큰 임베딩에 더해서 대규모 언어 모델을 위한 입력 임베딩 만들기
* 바이트 페어 인코더의 어휘 사전 크기 구하기
* 입력 토큰 256차원의 벡터표현으로 인코딩

In [75]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

* 데이터 로더에서 데이터를 샘플링한 다음 배치에 있는 각 샘플의 토큰을 256차원 벡터로 임베딩할 수 있다.
* 배치 크기가 8이고 샘플마다 4개의 토큰이 있다면 8 x 4 256 크기의 텐서가 만들어진다.

In [76]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [77]:
print("토큰 ID:\n", inputs)
print("\n입력 크기:\n", inputs.shape)

토큰 ID:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

입력 크기:
 torch.Size([8, 4])


In [78]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# 임베딩 벡터의 값을 확인합니다.
print(token_embeddings)

torch.Size([8, 4, 256])
tensor([[[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735, -0.1445],
         [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991, -1.1844],
         [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737, -0.0528],
         [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,  0.3483]],

        [[ 1.5460,  1.7368, -0.7848,  ..., -0.1004,  0.8584, -0.3421],
         [-1.8622, -0.1914, -0.3812,  ...,  1.1220, -0.3496,  0.6091],
         [ 1.9847, -0.6483, -0.1415,  ..., -0.3841, -0.9355,  1.4478],
         [ 0.9647,  1.2974, -1.6207,  ...,  1.1463,  1.5797,  0.3969]],

        [[-0.7713,  0.6572,  0.1663,  ..., -0.8044,  0.0542,  0.7426],
         [ 0.8046,  0.5047,  1.2922,  ...,  1.4648,  0.4097,  0.3205],
         [ 0.0795, -1.7636,  0.5750,  ...,  2.1823,  1.8231, -0.3635],
         [ 0.4267, -0.0647,  0.5686,  ..., -0.5209,  1.3065,  0.8473]],

        ...,

        [[-1.6156,  0.9610, -2.6437,  ..., -0.9645,  1.0888,  1.6383],
         [-0.3985, -0.9235, -1.31

* GPT-2는 절대 위치 임베딩을 사용하므로 또 다른 임베딩 층을 만들면 됩니다:


In [79]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# 임베딩 층의 가중치를 확인합니다.
print(pos_embedding_layer.weight)

Parameter containing:
tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       requires_grad=True)


In [80]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# 위치 임베딩 값을 확인합니다.
print(pos_embeddings)

torch.Size([4, 256])
tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       grad_fn=<EmbeddingBackward0>)


LLM에 사용될 입력 임베딩을 만들기 위해 토큰 임베딩과 위치 임베딩을 더한다.

In [81]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# 입력 임베딩 값을 확인합니다.
print(input_embeddings)

torch.Size([8, 4, 256])
tensor([[[ 2.2288,  0.5619,  0.8286,  ..., -0.6272, -0.2987,  0.8900],
         [ 2.0903, -0.4664, -0.0593,  ...,  0.9115, -1.0493, -1.6473],
         [-0.7158, -0.8304,  1.2494,  ...,  2.3952,  1.8773,  0.8051],
         [ 0.2703,  0.4029,  3.0514,  ...,  0.3595, -1.4548,  0.8310]],

        [[ 3.2835,  1.1749, -1.4150,  ..., -0.3281,  2.4332,  0.6924],
         [-0.2199, -0.9114, -0.1750,  ...,  1.5337, -0.1998,  0.1462],
         [ 1.5197, -1.4240,  0.4391,  ...,  1.0494, -1.4318,  2.3057],
         [ 0.2893,  0.8346, -0.1884,  ...,  1.9602,  0.8709,  0.8796]],

        [[ 0.9662,  0.0952, -0.4640,  ..., -1.0320,  1.6290,  1.7771],
         [ 2.4468, -0.2154,  1.4984,  ...,  1.8766,  0.5595, -0.1423],
         [-0.3856, -2.5393,  1.1556,  ...,  3.6157,  1.3267,  0.4944],
         [-0.2487, -0.5275,  2.0009,  ...,  0.2930,  0.5977,  1.3300]],

        ...,

        [[ 0.1219,  0.3991, -3.2740,  ..., -1.1921,  2.6637,  2.6728],
         [ 1.2438, -1.6436, -1.11

* 입력 전처리 워크플로의 초기 단계에서 입력테스트를 별개의 토큰으로 분할한다.
* 이 분할 단계 다음에 사전에 정의된 어휘 사전을 기반으로 토큰을 토큰 ID로 변환한다.